In [1]:
from numpy import append
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import os
from os.path import expanduser
from pathlib import Path
from airflow.models import DAG
from datetime import datetime
from datetime import timedelta
from airflow.operators import BashOperator

default_args = {
    'owner': 'balany1',
    'depends_on_past': False,
    'email': ['andrewmcnamara@live.co.uk'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'start_date': datetime(2023, 2, 8), # If you set a datetime previous to the curernt date, it will try to backfill
    'retry_delay': timedelta(minutes=5),
    'end_date': datetime(2024, 1, 1),

class SporcleAutobot():

    def __init__(self) -> None:
        super().__init__()
        self.driver = webdriver.Chrome(ChromeDriverManager().install())

    
    def load_page(self):
        '''Opens the Meatloaf page'''
        self.driver.get("https://www.sporcle.com/games/armeenrashid/what-is-the-only-thing-a-meatloaf-wont-do-for-love")
        time.sleep(3)

    def login(self):
        '''logs user in if not already done so'''
        try:
            sign_in_button = self.driver.find_element(by=By.XPATH, value="//*[@id='user-not-logged-in']")
            sign_in_button.click()
        except NoSuchElementException:
            pass
        
        time.sleep(3)
        self.driver.find_element(by=By.XPATH, value="/html/body/div[12]/div/div/div/div[3]")
        email_address = self.driver.find_element(by=By.XPATH, value="//*[@id='email']")
        email_address.send_keys("andrewmcnamara@live.co.uk")
        password = self.driver.find_element(by=By.XPATH, value="//*[@id='password']")
        password.send_keys("T00narmyspc")
        login_button = self.driver.find_element(by=By.XPATH, value="//*[@id='log-in-button']")
        login_button.click()
    

    def accept_cookies(self):
        '''Switches to cookies frame and clicks the accept button'''
        accept_cookies_frame = self.driver.switch_to.frame(self.driver.find_element(by= By.XPATH, value = "//*[@id='sp_message_iframe_756623']"))
        accept_cookies = self.driver.find_element(by=By.XPATH, value="//*[@id='notice']/div[5]/button[2]")
        accept_cookies.click()

    def play_game(self):
        
        time.sleep(3)
        play_button = self.driver.find_element(by=By.XPATH, value="//*[@id='button-play']")
        play_button.click()
        time.sleep(3)


    def find_answer(self):
        '''Clicks the button(s) required to finish the quiz'''
        time.sleep(3)
        answer_button = self.driver.find_element(by=By.XPATH, value = "//*[@id='slot0']/div")
        answer_button.click()

bot = SporcleAutobot()

with DAG(dag_id='sporcle_bot',
         default_args=default_args,
         schedule_interval='09 13 * * *',
         catchup=False,
         tags=['sporcle']
         ) as dag:
    # define task
    load_page = PythonOperator(
        task_id = 'load_page',
        python_callable = Batch.spark,
        )

if __name__ == "__main__":
    bot = SporcleAutobot()bot = SporcleAutobot()
    bot.load_page()
    bot.accept_cookies()
    bot.login()
    bot.play_game()
    bot.find_answer()
    bot.driver.quit()
    

/tmp/ipykernel_70436/894451313.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())
